Для курсового проекта я выбрала тему - "Heart Failure"
Для этого взяла дата-сет "Heart Failurer"
https://www.kaggle.com/fedesoriano/heart-failure-prediction

In [1]:
import pandas as pd
import numpy as np
import dill
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


import itertools
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, \
        roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import catboost as catb
%matplotlib inline

In [43]:
# input
DATASET_PATH = '../course_project/course_project_train.csv'

TEST_DATASET_PATH = 'heart.csv'


# output
TRAIN_X_FULL_PATH = 'hd_X_train.csv'
TRAIN_Y_FULL_PATH = "hd_y_train.csv"

TEST_X_FULL_PATH = "hd_X_test.csv"
TEST_Y_FULL_PATH = "hd_y_test.csv"

PIPELINE_FILE_PATH = "./models/heart_decease_pipeline.dill"


In [3]:
df = pd.read_csv(TEST_DATASET_PATH)
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


В качестве целевой переменной мы возьмем HeartDisease - . 

In [6]:
df['HeartDisease'].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

Наши классы достаточно сбалансированы

### Обзор переменных

In [7]:
df['Sex'].value_counts()

M    725
F    193
Name: Sex, dtype: int64

In [8]:
df['ChestPainType'].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

In [9]:
df['RestingECG'].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [10]:
df['ExerciseAngina'].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

In [11]:
df['ST_Slope'].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


На основе FeatureImportance пробной модели сократим количеством признаков, на которых обучается модель. уберем RestingECG, RestingBP, MaxHR, FastingBS

In [13]:
df.drop(axis=1, columns = ['RestingECG', 'RestingBP', 'MaxHR', 'FastingBS'], inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   Cholesterol     918 non-null    int64  
 4   ExerciseAngina  918 non-null    object 
 5   Oldpeak         918 non-null    float64
 6   ST_Slope        918 non-null    object 
 7   HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 57.5+ KB


In [15]:
df["Age"] = df["Age"].astype('int32')
df["Cholesterol"] = df["Cholesterol"].astype('int32')
df["HeartDisease"] = df["HeartDisease"].astype('int32')

In [16]:
cat_features = ['Sex','ChestPainType', 'ExerciseAngina', 'ST_Slope']

In [17]:
continuous_features = ['Age','Cholesterol','Oldpeak']

In [18]:
target = 'HeartDisease'

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [20]:
final_transformers = list()

for cat_col in cat_features:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_features:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [21]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

Добавим модель. В первом варианте будет использовать RandomForestClassifier

In [22]:
from sklearn.ensemble import RandomForestClassifier

pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

Теперь разделим наш датасет на тестовую и тренировочную выборки

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[target]), df[target],test_size=0.25,
                               shuffle=True, random_state=42)


In [24]:
#save test
X_test.to_csv(TEST_X_FULL_PATH, index=None)
y_test.to_csv(TEST_Y_FULL_PATH, index=None)
#save train
X_train.to_csv(TRAIN_X_FULL_PATH, index=None)
y_train.to_csv(TRAIN_Y_FULL_PATH, index=None)


### Обучение модели

In [25]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [26]:
from sklearn.model_selection import GridSearchCV

params_rf={'classifier__max_features':[0.3, 0.5, 0.7],
        'classifier__min_samples_leaf':[1, 2, 3],
        'classifier__max_depth':[3,5,7]
        }

In [27]:
%time
grid_rf = GridSearchCV(pipeline_1,
                    param_grid=params_rf,
                    cv=6,
                    refit=False)

search_rf = grid_rf.fit(X_train, y_train)
search_rf.best_params_

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.1 µs


{'classifier__max_depth': 7,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 1}

In [28]:
pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth = 7,\
                                          max_features=0.3, \
                                          random_state = 42,\
                                         min_samples_leaf =2)),
])

In [29]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


В качестве второго варианта для модели возьмем CatBoost

In [30]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(random_state = 42)),
])

In [31]:
params_cb={'classifier__iterations':[50, 100, 200],
        'classifier__max_depth':[3,5,7]
        }

In [32]:
%time
grid_cb = GridSearchCV(pipeline_2,
                    param_grid=params_cb,
                    cv=6,
                    refit=False)

search_cb = grid_cb.fit(X_train, y_train)


CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 13.1 µs
Learning rate set to 0.126681
0:	learn: 0.6396506	total: 58.3ms	remaining: 2.86s
1:	learn: 0.5999934	total: 60.5ms	remaining: 1.45s
2:	learn: 0.5612588	total: 62.7ms	remaining: 982ms
3:	learn: 0.5313902	total: 64.7ms	remaining: 743ms
4:	learn: 0.5058978	total: 66.1ms	remaining: 595ms
5:	learn: 0.4860302	total: 67.8ms	remaining: 497ms
6:	learn: 0.4696593	total: 69.3ms	remaining: 426ms
7:	learn: 0.4533983	total: 71.3ms	remaining: 374ms
8:	learn: 0.4384939	total: 73.4ms	remaining: 334ms
9:	learn: 0.4269121	total: 75.4ms	remaining: 301ms
10:	learn: 0.4148465	total: 77.6ms	remaining: 275ms
11:	learn: 0.4075709	total: 79.7ms	remaining: 253ms
12:	learn: 0.3992330	total: 81.3ms	remaining: 231ms
13:	learn: 0.3926396	total: 82.7ms	remaining: 213ms
14:	learn: 0.3832423	total: 84ms	remaining: 196ms
15:	learn: 0.3768977	total: 86ms	remaining: 183ms
16:	learn: 0.3713167	total: 87.5ms	remaining: 170ms
17:	learn: 0.3676245	total: 89ms

4:	learn: 0.5086329	total: 6.29ms	remaining: 56.6ms
5:	learn: 0.4845370	total: 8.32ms	remaining: 61ms
6:	learn: 0.4680660	total: 9.79ms	remaining: 60.1ms
7:	learn: 0.4510282	total: 10.9ms	remaining: 57.3ms
8:	learn: 0.4359598	total: 11.9ms	remaining: 54ms
9:	learn: 0.4214754	total: 12.7ms	remaining: 50.8ms
10:	learn: 0.4095645	total: 18.1ms	remaining: 64.3ms
11:	learn: 0.3999974	total: 19.4ms	remaining: 61.3ms
12:	learn: 0.3924466	total: 20.4ms	remaining: 58.1ms
13:	learn: 0.3844347	total: 21.5ms	remaining: 55.3ms
14:	learn: 0.3769801	total: 22.6ms	remaining: 52.7ms
15:	learn: 0.3708618	total: 23.7ms	remaining: 50.3ms
16:	learn: 0.3661935	total: 29.5ms	remaining: 57.3ms
17:	learn: 0.3629837	total: 35.3ms	remaining: 62.8ms
18:	learn: 0.3568490	total: 36.5ms	remaining: 59.5ms
19:	learn: 0.3531297	total: 37.8ms	remaining: 56.7ms
20:	learn: 0.3499295	total: 39ms	remaining: 53.8ms
21:	learn: 0.3467993	total: 39.8ms	remaining: 50.7ms
22:	learn: 0.3442255	total: 40.8ms	remaining: 47.9ms
23:	l

Learning rate set to 0.126681
0:	learn: 0.6396167	total: 941us	remaining: 46.1ms
1:	learn: 0.5929621	total: 1.94ms	remaining: 46.5ms
2:	learn: 0.5577495	total: 3.05ms	remaining: 47.9ms
3:	learn: 0.5274785	total: 4.07ms	remaining: 46.8ms
4:	learn: 0.5022701	total: 4.65ms	remaining: 41.9ms
5:	learn: 0.4802736	total: 5.64ms	remaining: 41.3ms
6:	learn: 0.4604199	total: 6.52ms	remaining: 40.1ms
7:	learn: 0.4432006	total: 7.45ms	remaining: 39.1ms
8:	learn: 0.4255825	total: 8.39ms	remaining: 38.2ms
9:	learn: 0.4126185	total: 9.29ms	remaining: 37.2ms
10:	learn: 0.4011442	total: 10.6ms	remaining: 37.7ms
11:	learn: 0.3912550	total: 11.9ms	remaining: 37.6ms
12:	learn: 0.3823729	total: 12.9ms	remaining: 36.7ms
13:	learn: 0.3744156	total: 14ms	remaining: 36ms
14:	learn: 0.3681859	total: 15.3ms	remaining: 35.8ms
15:	learn: 0.3618245	total: 16.6ms	remaining: 35.2ms
16:	learn: 0.3565566	total: 18ms	remaining: 35ms
17:	learn: 0.3502541	total: 19.7ms	remaining: 35.1ms
18:	learn: 0.3445628	total: 21.8ms	

Learning rate set to 0.126681
0:	learn: 0.6466418	total: 2.51ms	remaining: 123ms
1:	learn: 0.6040817	total: 4.71ms	remaining: 113ms
2:	learn: 0.5687458	total: 7.37ms	remaining: 116ms
3:	learn: 0.5407868	total: 10.5ms	remaining: 121ms
4:	learn: 0.5162211	total: 12.6ms	remaining: 114ms
5:	learn: 0.4937301	total: 15.6ms	remaining: 115ms
6:	learn: 0.4729338	total: 18.1ms	remaining: 111ms
7:	learn: 0.4573254	total: 20.5ms	remaining: 108ms
8:	learn: 0.4424758	total: 23.2ms	remaining: 106ms
9:	learn: 0.4271510	total: 25.5ms	remaining: 102ms
10:	learn: 0.4139086	total: 28.3ms	remaining: 100ms
11:	learn: 0.4028252	total: 31.4ms	remaining: 99.3ms
12:	learn: 0.3934778	total: 33.8ms	remaining: 96.1ms
13:	learn: 0.3849816	total: 36.5ms	remaining: 94ms
14:	learn: 0.3770918	total: 40.7ms	remaining: 95.1ms
15:	learn: 0.3703814	total: 43.2ms	remaining: 91.9ms
16:	learn: 0.3640294	total: 45.7ms	remaining: 88.6ms
17:	learn: 0.3573388	total: 47.7ms	remaining: 84.8ms
18:	learn: 0.3517510	total: 50.5ms	rema

Learning rate set to 0.126775
0:	learn: 0.6431205	total: 4.5ms	remaining: 220ms
1:	learn: 0.5995766	total: 7.48ms	remaining: 180ms
2:	learn: 0.5648903	total: 10.9ms	remaining: 171ms
3:	learn: 0.5358209	total: 14.6ms	remaining: 168ms
4:	learn: 0.5119562	total: 17.5ms	remaining: 158ms
5:	learn: 0.4890186	total: 20.1ms	remaining: 147ms
6:	learn: 0.4689138	total: 23.4ms	remaining: 144ms
7:	learn: 0.4525318	total: 26ms	remaining: 137ms
8:	learn: 0.4375200	total: 30.7ms	remaining: 140ms
9:	learn: 0.4233948	total: 33.8ms	remaining: 135ms
10:	learn: 0.4101996	total: 36.4ms	remaining: 129ms
11:	learn: 0.3987608	total: 39.1ms	remaining: 124ms
12:	learn: 0.3893525	total: 41.5ms	remaining: 118ms
13:	learn: 0.3819172	total: 44.1ms	remaining: 113ms
14:	learn: 0.3733138	total: 46.7ms	remaining: 109ms
15:	learn: 0.3653183	total: 49.1ms	remaining: 104ms
16:	learn: 0.3590309	total: 51.6ms	remaining: 100ms
17:	learn: 0.3534935	total: 54ms	remaining: 96.1ms
18:	learn: 0.3464861	total: 56.5ms	remaining: 92

...

188:	learn: 0.1396287	total: 559ms	remaining: 32.5ms
189:	learn: 0.1389934	total: 566ms	remaining: 29.8ms
190:	learn: 0.1380017	total: 569ms	remaining: 26.8ms
191:	learn: 0.1373729	total: 572ms	remaining: 23.8ms
192:	learn: 0.1368543	total: 574ms	remaining: 20.8ms
193:	learn: 0.1357931	total: 576ms	remaining: 17.8ms
194:	learn: 0.1354509	total: 579ms	remaining: 14.8ms
195:	learn: 0.1350740	total: 582ms	remaining: 11.9ms
196:	learn: 0.1343621	total: 584ms	remaining: 8.89ms
197:	learn: 0.1338837	total: 586ms	remaining: 5.92ms
198:	learn: 0.1326241	total: 589ms	remaining: 2.96ms
199:	learn: 0.1322856	total: 592ms	remaining: 0us
Learning rate set to 0.035532
0:	learn: 0.6544182	total: 2.4ms	remaining: 478ms
1:	learn: 0.6274836	total: 4.78ms	remaining: 473ms
2:	learn: 0.6037289	total: 7.34ms	remaining: 482ms
3:	learn: 0.5782218	total: 10ms	remaining: 491ms
4:	learn: 0.5555703	total: 13.3ms	remaining: 518ms
5:	learn: 0.5367222	total: 16ms	remaining: 519ms
6:	learn: 0.5175861	total: 19.1ms	re

189:	learn: 0.1339706	total: 598ms	remaining: 31.5ms
190:	learn: 0.1334862	total: 603ms	remaining: 28.4ms
191:	learn: 0.1327705	total: 609ms	remaining: 25.4ms
192:	learn: 0.1321056	total: 615ms	remaining: 22.3ms
193:	learn: 0.1315442	total: 619ms	remaining: 19.2ms
194:	learn: 0.1311185	total: 624ms	remaining: 16ms
195:	learn: 0.1307530	total: 627ms	remaining: 12.8ms
196:	learn: 0.1305234	total: 631ms	remaining: 9.61ms
197:	learn: 0.1298359	total: 635ms	remaining: 6.42ms
198:	learn: 0.1290440	total: 638ms	remaining: 3.21ms
199:	learn: 0.1283389	total: 641ms	remaining: 0us
Learning rate set to 0.035532
0:	learn: 0.6597814	total: 15ms	remaining: 2.99s
1:	learn: 0.6291724	total: 18.8ms	remaining: 1.86s
2:	learn: 0.6052433	total: 21.7ms	remaining: 1.43s
3:	learn: 0.5795869	total: 24.4ms	remaining: 1.2s
4:	learn: 0.5624141	total: 26.5ms	remaining: 1.03s
5:	learn: 0.5397073	total: 29.4ms	remaining: 950ms
6:	learn: 0.5193212	total: 32.2ms	remaining: 888ms
7:	learn: 0.5020683	total: 34.3ms	rema

152:	learn: 0.1532285	total: 506ms	remaining: 155ms
153:	learn: 0.1525307	total: 510ms	remaining: 152ms
154:	learn: 0.1515351	total: 515ms	remaining: 150ms
155:	learn: 0.1509929	total: 520ms	remaining: 147ms
156:	learn: 0.1497065	total: 525ms	remaining: 144ms
157:	learn: 0.1490132	total: 540ms	remaining: 144ms
158:	learn: 0.1483745	total: 546ms	remaining: 141ms
159:	learn: 0.1481510	total: 551ms	remaining: 138ms
160:	learn: 0.1469315	total: 554ms	remaining: 134ms
161:	learn: 0.1469187	total: 556ms	remaining: 130ms
162:	learn: 0.1463227	total: 560ms	remaining: 127ms
163:	learn: 0.1456156	total: 565ms	remaining: 124ms
164:	learn: 0.1448928	total: 569ms	remaining: 121ms
165:	learn: 0.1445002	total: 572ms	remaining: 117ms
166:	learn: 0.1439837	total: 576ms	remaining: 114ms
167:	learn: 0.1432487	total: 580ms	remaining: 110ms
168:	learn: 0.1425987	total: 583ms	remaining: 107ms
169:	learn: 0.1425481	total: 585ms	remaining: 103ms
170:	learn: 0.1418642	total: 588ms	remaining: 99.8ms
171:	learn:

146:	learn: 0.1643173	total: 502ms	remaining: 181ms
147:	learn: 0.1636185	total: 514ms	remaining: 180ms
148:	learn: 0.1631827	total: 519ms	remaining: 178ms
149:	learn: 0.1629154	total: 526ms	remaining: 175ms
150:	learn: 0.1625260	total: 530ms	remaining: 172ms
151:	learn: 0.1619741	total: 532ms	remaining: 168ms
152:	learn: 0.1619025	total: 534ms	remaining: 164ms
153:	learn: 0.1612046	total: 537ms	remaining: 160ms
154:	learn: 0.1605794	total: 540ms	remaining: 157ms
155:	learn: 0.1597028	total: 543ms	remaining: 153ms
156:	learn: 0.1594157	total: 546ms	remaining: 150ms
157:	learn: 0.1589410	total: 549ms	remaining: 146ms
158:	learn: 0.1588568	total: 551ms	remaining: 142ms
159:	learn: 0.1584826	total: 554ms	remaining: 138ms
160:	learn: 0.1576931	total: 556ms	remaining: 135ms
161:	learn: 0.1569530	total: 559ms	remaining: 131ms
162:	learn: 0.1566272	total: 564ms	remaining: 128ms
163:	learn: 0.1562640	total: 568ms	remaining: 125ms
164:	learn: 0.1556012	total: 572ms	remaining: 121ms
165:	learn: 

135:	learn: 0.1540218	total: 366ms	remaining: 172ms
136:	learn: 0.1526105	total: 373ms	remaining: 172ms
137:	learn: 0.1520501	total: 381ms	remaining: 171ms
138:	learn: 0.1515297	total: 387ms	remaining: 170ms
139:	learn: 0.1509860	total: 391ms	remaining: 167ms
140:	learn: 0.1500062	total: 397ms	remaining: 166ms
141:	learn: 0.1488363	total: 412ms	remaining: 168ms
142:	learn: 0.1476088	total: 415ms	remaining: 166ms
143:	learn: 0.1469936	total: 419ms	remaining: 163ms
144:	learn: 0.1464675	total: 425ms	remaining: 161ms
145:	learn: 0.1460972	total: 430ms	remaining: 159ms
146:	learn: 0.1459246	total: 433ms	remaining: 156ms
147:	learn: 0.1457490	total: 436ms	remaining: 153ms
148:	learn: 0.1450807	total: 439ms	remaining: 150ms
149:	learn: 0.1445622	total: 441ms	remaining: 147ms
150:	learn: 0.1439707	total: 444ms	remaining: 144ms
151:	learn: 0.1437993	total: 446ms	remaining: 141ms
152:	learn: 0.1433448	total: 449ms	remaining: 138ms
153:	learn: 0.1426868	total: 453ms	remaining: 135ms
154:	learn: 

In [33]:
search_cb.best_params_

{'classifier__iterations': 200, 'classifier__max_depth': 3}

In [34]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(iterations = 200,\
                                           max_depth=3,
                                           random_state = 42)),
])


In [35]:
pipeline_2.fit(X_train, y_train)

Learning rate set to 0.038418
0:	learn: 0.6648744	total: 1.01ms	remaining: 201ms
1:	learn: 0.6456676	total: 2.14ms	remaining: 212ms
2:	learn: 0.6242099	total: 3.17ms	remaining: 208ms
3:	learn: 0.6031473	total: 4.18ms	remaining: 205ms
4:	learn: 0.5828988	total: 5.29ms	remaining: 206ms
5:	learn: 0.5629649	total: 6.39ms	remaining: 207ms
6:	learn: 0.5437890	total: 7.62ms	remaining: 210ms
7:	learn: 0.5327356	total: 9.32ms	remaining: 224ms
8:	learn: 0.5207788	total: 10.4ms	remaining: 221ms
9:	learn: 0.5051044	total: 11.4ms	remaining: 217ms
10:	learn: 0.4957560	total: 12.4ms	remaining: 213ms
11:	learn: 0.4860590	total: 13.7ms	remaining: 214ms
12:	learn: 0.4752287	total: 14.9ms	remaining: 215ms
13:	learn: 0.4649557	total: 16.1ms	remaining: 214ms
14:	learn: 0.4578381	total: 17.2ms	remaining: 212ms
15:	learn: 0.4501429	total: 18.2ms	remaining: 209ms
16:	learn: 0.4403168	total: 19.2ms	remaining: 207ms
17:	learn: 0.4353034	total: 20.2ms	remaining: 204ms
18:	learn: 0.4276963	total: 21.3ms	remaining

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [36]:
predictions1 = pipeline_1.predict_proba(X_test)
predictions2 = pipeline_2.predict_proba(X_test)

In [37]:
def get_model_scores(y_test, y_score):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, y_score)
    return precision[ix], recall[ix], fscore[ix], roc_auc, thresholds[ix]

In [38]:
pr1, rec1, f1, roc1, thresh1 =  get_model_scores(y_test, predictions1[:, 1][:])
pr1, rec1, f1, roc1, thresh1 

(0.8854961832061069,
 0.8787878787878788,
 0.8821292775665399,
 0.9221938775510203,
 0.4288438592790484)

In [39]:
pr2, rec2, f2, roc2, thresh2 =  get_model_scores(y_test, predictions2[:, 1][:])
pr2, rec2, f2, roc2, thresh2 

(0.8776978417266187,
 0.9242424242424242,
 0.9003690036900369,
 0.9355674087816945,
 0.33601411138028925)

Выбираем pipeline_2(catboost), потому что у него метрики более высокие

In [40]:
def show_feature_importances(feature_names, feature_importances, get_top=None):
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
       
    plt.figure(figsize = (20, len(feature_importances) * 0.355))
    
    sns.barplot(feature_importances['importance'], feature_importances['feature'])
    
    plt.xlabel('Importance')
    plt.title('Importance of features')
    plt.show()
    
    if get_top is not None:
        return feature_importances['feature'][:get_top].tolist()

На основе FeatureImportance сократим количеством признаков, на которых обучается модель.
уберем RestingECG, RestingBP, MaxHR, FastingBS

In [41]:
model_dict = {
    'model':pipeline_2,\
    'version':'0.0.1'
    
}

In [44]:
with open(PIPELINE_FILE_PATH, "wb") as f:
    dill.dump(model_dict, f)